# Tutorial on Interfacing with GeoModels using PyVista and Trame

## Introduction

This Jupyter notebook serves as a guide to demonstrate how to leverage the powerful capabilities of PyVista, Trame for interfacing with GeoModels. These tools provide robust frameworks for rendering, manipulating, and analyzing 3D geological models using the Visualization Toolkit (VTK). PyVista offers some advantages over other visualization tools, with its ability to interface directly with the numpy arrays that store the core Geomodel data structures. Some purpose built displays modes have been developed for viewing Geomodels, but the full power of PyVista can be used to extend the library into more advanced custom visualization and analysis.

## Purpose

The purpose of this notebook is to provide a step-by-step tutorial on how to utilize PyVista to work with geological models. We will cover the following key areas:

1. **Introduction to PyVista, Trame**: Understanding the core functionalities and features of each library.
2. **Setting Up the Environment**: Instructions on how to install and configure the necessary libraries.
3. **Loading and Visualizing GeoModels**: Techniques for importing and visualizing geological models using PyVista.
4. **Interactive Visualization with Trame**: Utilizing Trame to create interactive web-based visualizations.

By the end of this notebook, you will have an understanding of how to effectively interface with GeoModels.

## Libraries Overview

- **PyVista**: A flexible and easy-to-use library for 3D plotting and mesh analysis based on VTK. It provides a high-level API for visualizing and analyzing 3D data.
  - [PyVista Documentation](https://docs.pyvista.org/)
  - [PyVista Tutorials](https://docs.pyvista.org/examples/index.html)
  - [PyVista Installation](https://docs.pyvista.org/getting-started/installation.html)

- **Trame**: A framework for building interactive web applications with VTK. It allows seamless integration of VTK-based visualizations into web interfaces, enabling interactive exploration of 3D models.
  - [Trame Documentation](https://kitware.github.io/trame/)

- **VTK (Visualization Toolkit)**: An open-source software system for 3D computer graphics, image processing, and visualization. VTK forms the backbone of both PyVista and Trame, offering extensive capabilities for 3D data processing and rendering.
  - [VTK Documentation](https://vtk.org/)

## Getting Started

To begin, ensure you have the necessary libraries installed. You can set up your environment by following the installation instructions provided in the PyVista installation instructions above. In addition, if planning to use Jupyter or html web embeddings for visualization you should install Trame as well by following the [PyVista Tutorials Getting Started](https://tutorial.pyvista.org/getting-started.html) for installation and [PyVista in Jupyter](https://tutorial.pyvista.org/tutorial/00_jupyter/index.html) for Jupyter notebook setup.

In [1]:
import pyvista as pv

from structgeo import model as geo
from structgeo import plot as geovis
from structgeo import probability as rv
from structgeo.plot import ModelGenerator

# Client backend deliver fast viewing performance
pv.set_jupyter_backend('client')
pv.set_plot_theme('default')

In [2]:
# Define a function to generate a model
def generate_model():
    # Start with some model specifications
    resolution = 128
    # Cubic model keeps 
    max_x = 10
    min_x = - max_x
    bounds = (min_x,max_x)
    model = geo.GeoModel(bounds, resolution)
    # Bedrock layer
    bedrock = geo.Bedrock(-5, 0)
    
    sb = geo.SedimentBuilder(1,6,2,8)
    sediment = geo.Sedimentation(*sb.get_layers())
    
    fault = geo.Fault(dip=63, rake = 32)
    sediment2 = geo.Sedimentation(*sb.get_layers())
    
    model.add_history([bedrock, sediment, fault, sediment2])
    model.compute_model()
    return model

model = generate_model()

The Geomodel holds data in a few different forms. It has three mesh grid arrays X, Y, Z forming the set of points, an xyz array that is $n\times3$ of the the $(x,y,z)$ points, and a final data array for the cell values of the rock. Some time snapshots that record the model states where rock assignments were applied are also stored in the Geomodel mesh_snapshots and data_snapshots. With the $0$ index position for the time of the snapshot.

In [3]:
import pandas as pd
# Collect shapes in a dictionary
shape_dict = {
    "model.X": model.X.shape,
    "model.Y": model.Y.shape,
    "model.Z": model.Z.shape,
    "model.xyz": model.xyz.shape,
    "model.data": model.data.shape,
    "model.mesh_snapshots": model.mesh_snapshots.shape,
    "model.data_snapshots": model.data_snapshots.shape
}

# Convert the dictionary to a pandas DataFrame
shape_df = pd.DataFrame(list(shape_dict.items()), columns=["Attribute", "Shape"])

# Display the DataFrame
print(shape_df)

              Attribute            Shape
0               model.X  (128, 128, 128)
1               model.Y  (128, 128, 128)
2               model.Z  (128, 128, 128)
3             model.xyz     (2097152, 3)
4            model.data       (2097152,)
5  model.mesh_snapshots  (2, 2097152, 3)
6  model.data_snapshots     (2, 2097152)


The data can be visualized with PyVista in a few different ways. A mesh approach over a structured grid assigns calculated data values aa point on an interconnected mesh. This differs from a true voxel representation since each of the sample points do not have any volume. The space between them is filled using interpolation which will produce non-categorical values.


In [4]:
# Meshgrid representation in Pyvista
mesh = pv.StructuredGrid(model.X, model.Y, model.Z)
# The data is not in the correct ordering though, so we shape back to 3D grid and then ravel to 1D
mesh['Rock Type'] = model.data.reshape(model.X.shape).ravel(order='F')
mesh = mesh.threshold(-1, all_scalars=True) # Filter out the nans

# Plot the model mesh with interpolated values across mesh cells
p = pv.Plotter(notebook=True)
p.add_mesh(mesh, scalars='Rock Type', categories=True )
p.add_bounding_box()
p.show(jupyter_backend='client')

Widget(value='<iframe src="http://localhost:50114/index.html?ui=P_0x22675159590_0&reconnect=auto" class="pyvis…

On a more complicated model, the interpolation becomes apparent and produces image artifacts.

In [5]:
import pickle as pkl
model = geo.GeoModel
with open('C:/Users/sghys/Summer2024/StructuralGeo/paper-image-generators/model_scale30_ar1_res128.pkl', 'rb') as f:
    model = pkl.load(f)

model.history[14].strike = 30  
print(model.get_history_string())
model.compute_model()


Geological History:
1: Bedrock: with z <= -1500.0 and value 0.0
2: Sedimentation: rock type values [1, 2, 3...], and thicknesses 343.925, 142.882, 626.886....
3: Fold: strike 109.5°, dip 32.3°, rake 268.0°, period 786.1,amplitude 7.5, origin (0.0, 0.0, 0.0).
4: Fold: strike 72.3°, dip 65.4°, rake 112.4°, period 782.4,amplitude 14.6, origin (0.0, 0.0, 0.0).
5: Fold: strike 200.7°, dip 9.1°, rake 73.5°, period 453.0,amplitude 13.6, origin (0.0, 0.0, 0.0).
6: Fold: strike 144.0°, dip 118.3°, rake 119.2°, period 618.4,amplitude 14.3, origin (0.0, 0.0, 0.0).
7: Fold: strike 30.0°, dip 70.0°, rake 15.0°, period 4000.0,amplitude 500.0, origin (0.0, 0.0, 0.0).
8: UnconformityBase: base 0.0, value nan
9: Fold: strike 30.0°, dip 70.0°, rake 15.0°, period 4000.0,amplitude -200.0, origin (0.0, 0.0, 0.0).
10: Dike: strike 35.0°, dip 50.0°, width 10.0, origin (100.0, 200.0, 0.0), value 8.0.
11: Dike: strike 37.0°, dip 47.0°, width 30.0, origin (100.0, -100.0, 0.0), value 9.0.
12: Dike: strike 39.0°,

In [6]:
# Meshgrid representation in Pyvista
mesh = pv.StructuredGrid(model.X, model.Y, model.Z)
# The data is not in the correct ordering though, so we shape back to 3D grid and then ravel to 1D
mesh['Rock Type'] = model.data.reshape(model.X.shape).ravel(order='F')
mesh = mesh.threshold(-1, all_scalars=True) # Filter out the nans

# Plot the model mesh with interpolated values across mesh cells
p = pv.Plotter(notebook=True)
p.add_mesh(mesh, scalars='Rock Type', categories=True)
p.add_bounding_box()
p.show(jupyter_backend='client')

Widget(value='<iframe src="http://localhost:50114/index.html?ui=P_0x22672f15a10_1&reconnect=auto" class="pyvis…

A solution is to instead define a grid of *cells* where the data is assigned to a volume instead of a point. This is a voxel representation and does not require any interpolation to fill the volume. However, the cell array needs to be constructed from the model parameters. Note that when assigning data to a PyVista datatype, it can automatically deduce it it is for the points or the cells by matching the array size to the number of points or cells in the mesh.

In [7]:
# Create a padded grid with n+1 nodes and node spacing equal to model sample spacing    
dimensions = tuple(x + 1 for x in model.resolution)
spacing = tuple((x[1] - x[0])/(r-1) for x,r in zip(model.bounds, model.resolution))
# pad origin with a half cell size to center the grid
origin = tuple(x[0] - cs/2 for x,cs in zip(model.bounds, spacing))
# Create a structured grid with n+1 nodes in each dimension forming n^3 cells
grid = pv.ImageData(
    dimensions = dimensions,
    spacing = spacing,
    origin = origin,
)    
# Necessary to reshape data vector in Fortran order to match the grid
grid['values'] = model.data.reshape(model.resolution).ravel(order='F')
grid = grid.threshold(-.5, all_scalars=True)

# The same functionality can be achieved with a library call
grid = geovis.get_voxel_grid_from_model(model, threshold=-.5)

p = pv.Plotter()
p.add_mesh(grid, scalars='values', categories=True)
p.add_bounding_box()
p.show()

Widget(value='<iframe src="http://localhost:50114/index.html?ui=P_0x22607cbba50_2&reconnect=auto" class="pyvis…

In [8]:
model

Parameter,Value
Name,model
Data Type,
Bounds,"((-1920, 1920), (-1920, 1920), (-1920, 1920))"
Resolution,"(128, 128, 128)"
History,"Bedrock: with z <= -1500.0 and value 0.0Sedimentation: rock type values [1, 2, 3...], and thicknesses 343.925, 142.882, 626.886....Fold: strike 109.5°, dip 32.3°, rake 268.0°, period 786.1,amplitude 7.5, origin (0.0, 0.0, 0.0).Fold: strike 72.3°, dip 65.4°, rake 112.4°, period 782.4,amplitude 14.6, origin (0.0, 0.0, 0.0).Fold: strike 200.7°, dip 9.1°, rake 73.5°, period 453.0,amplitude 13.6, origin (0.0, 0.0, 0.0).Fold: strike 144.0°, dip 118.3°, rake 119.2°, period 618.4,amplitude 14.3, origin (0.0, 0.0, 0.0).Fold: strike 30.0°, dip 70.0°, rake 15.0°, period 4000.0,amplitude 500.0, origin (0.0, 0.0, 0.0).UnconformityBase: base 0.0, value nanFold: strike 30.0°, dip 70.0°, rake 15.0°, period 4000.0,amplitude -200.0, origin (0.0, 0.0, 0.0).Dike: strike 35.0°, dip 50.0°, width 10.0, origin (100.0, 200.0, 0.0), value 8.0.Dike: strike 37.0°, dip 47.0°, width 30.0, origin (100.0, -100.0, 0.0), value 9.0.Dike: strike 39.0°, dip 44.0°, width 50.0, origin (100.0, -489.2, 0.0), value 10.0.Dike: strike 41.0°, dip 41.0°, width 70.0, origin (100.0, -921.2, 0.0), value 11.0.Dike: strike 43.0°, dip 38.0°, width 90.0, origin (100.0, -1383.4, 0.0), value 12.0.Fault with strike 1718.9°, dip 70.0°, rake 45.0°, amplitude 500.0, origin (0, 0, 0).Tilt: strike -40.0°, dip 8.0°,origin (0.0, 0.0, 0.0)Sedimentation: rock type values [13, 14, 15], and thicknesses 155.000, 145.000, 110.000.Fold: strike 120.0°, dip 0.0°, rake 15.0°, period 8000.0,amplitude -500.0, origin (0.0, 0.0, 0.0)."
